In [95]:
import pandas as pd
#import dask.dataframe as pd
import time
import numpy as np
from contextlib import contextmanager
import os; os.environ['OMP_NUM_THREADS'] = '4'
import dask.dataframe as dd

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.1f} s')

dtypes = {
        'ip':'uint32',
        'app': 'uint16',
        'device': 'uint16',
        'os': 'uint16',
        'channel': 'uint16',
        'is_attributed': 'uint8'
        }
    
path = '../input/'
NROWS = int(9308567)
TRAINFILE = 'train06.csv'
#TRAINFILE = "train_sample.csv"


with timer('read Pandas data'):
    train = pd.read_csv(path+TRAINFILE, skiprows=0, nrows=NROWS,
                        dtype=dtypes)
    train.columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time', 'is_attributed']
    train = train.drop(['attributed_time'], axis=1)
    train['freqip'] = train.groupby(['ip'])['ip'].transform('count')
    train['freqipdev'] = train.groupby(['ip', 'device'])['ip'].transform('count')
    
with timer('read Dask data'):
    dask_df = dd.read_csv(path+TRAINFILE, dtype=dtypes).head(NROWS, npartitions=-1)
    dask_df.columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time', 'is_attributed']
    dask_df = dask_df.drop(['attributed_time'], axis=1)
    dask_df['freqip'] = dask_df.groupby(['ip'])['ip'].transform('count')
    dask_df['freqipdev'] = dask_df.groupby(['ip', 'device'])['ip'].transform('count')

[read Pandas data] done in 14.4 s
[read Dask data] done in 11.0 s


In [94]:
dask_df.head()

,ip,app,device,os,channel,click_time,is_attributed,freqip,freqipdev
0,17357,3,1,19,379,2017-11-06 14:33:34,0,927,922
1,35810,3,1,13,379,2017-11-06 14:34:12,0,424,375
2,45745,14,1,13,478,2017-11-06 14:34:52,0,8949,8075
3,161007,3,1,13,379,2017-11-06 14:35:08,0,176,176
4,18787,3,1,16,379,2017-11-06 14:36:26,0,198,198


In [16]:

dask_df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
1,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
2,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
3,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0
4,18787,3,1,16,379,2017-11-06 14:36:26,NaN,0


In [19]:
dask_df.groupby(['ip', 'device']).count()

app   os  channel  click_time  attributed_time  is_attributed
ip     device                                                               
9      1       150  150      150         150                0            150
10     1        49   49       49          49                0             49
19     1         7    7        7           7                0              7
20     0         1    1        1           1                0              1
       1       128  128      128         128                0            128
       3032      2    2        2           2                0              2
25     0         1    1        1           1                0              1
       1        51   51       51          51                1             51
27     1       212  212      212         212                0            212
       3032     17   17       17          17                0             17
31     0         1    1        1           1                0              1
       1        36   36       36          36                0             36
       3032      1    1        1           1                0              1
32     1        22   22       22          22                0             22
36     1       367  367      367         367                0            367
       3032     45   45       45          45                0             45
39     0         1    1        1           1                1              1
       1        26   26       26          26                0             26
45     1       157  157      157         157                0            157
52     1        46   46       46          46                0             46
59     1       129  129      129         129                0            129
       3032     23   23       23          23                0             23
60     1        47   47       47          47                0             47
       3032      3    3        3           3                0              3
61     1        48   48       48          48                0             48
63     1        92   92       92          92                0             92
       3032      7    7        7           7                0              7
81     1        77   77       77          77                0             77
83     1        26   26       26          26                0             26
85     1        68   68       68          68                0             68
...            ...  ...      ...         ...              ...            ...
212725 1        66   66       66          66                0             66
212726 1        40   40       40          40                0             40
212728 1        60   60       60          60                0             60
212731 1         3    3        3           3                0              3
       2         1    1        1           1                0              1
212734 1        10   10       10          10                1             10
212737 1       132  132      132         132                0            132
       2         1    1        1           1                0              1
212740 0         1    1        1           1                1              1
       1         1    1        1           1                0              1
212742 1       184  184      184         184                0            184
       2         2    2        2           2                0              2
       3032      1    1        1           1                0              1
212743 1        43   43       43          43                0             43
       2         2    2        2           2                0              2
212745 1         2    2        2           2                0              2
212746 1        11   11       11          11                1             11
212752 1         2    2        2           2                0              2
212753 230       1    1        1           1                1              1
212754 1   